In [ ]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data_dir = '/content/drive/My Drive/Colab Notebooks/jigsaw-toxic-comment-classification'

# Full path to the train.csv file
train_file_path = os.path.join(data_dir, 'train.csv')
print(train_file_path)


In [ ]:
print(os.listdir(data_dir))

In [ ]:
train_file_path = '/content/drive/MyDrive/Colab Notebooks/jigsaw-toxic-comment-classification/train.csv/train.csv'
print(os.path.isfile(train_file_path))  # This should print True if the file exists


Read csv file data to pandas DataFrame

In [ ]:
df=pd.read_csv(train_file_path)

Display first 10 rows of DataFrame or dataset

In [ ]:
df.head()

Display Last 10 rows of DataFrame or dataset

In [ ]:
df.tail()

used to access a specific entry in a pandas DataFrame
index value 159489

In [ ]:
df.iloc[159489]['comment_text']

used to select a specific row from a subset of columns in a pandas DataFrame

In [ ]:
df[df.columns[2:]].iloc[6]

used to filter rows in a pandas DataFrame where the value in the toxic column is equal to 1

In [ ]:
df[df['toxic']==1]

Calculates number of Toxic comments in the DataFrame


In [ ]:
total_toxic_comments = df[df['toxic'] == True].shape[0]
print(f"Total count of toxic comments: {total_toxic_comments}")

In [ ]:
from tensorflow.keras.layers import TextVectorization

In [ ]:
x=df['comment_text']
y=df[df.columns[2:]].values
print(x)


In [ ]:
print(y)

In [ ]:
MAX_FEATURES=200000 #number of words in vocab

The below code helps to convert text data to numberical values for the input to the model.
It sets the max-vocabulary words to be used, fixes the length of the sentence and defines the outptu to be int format.

In [ ]:
#initailize the text vectorization layer
vectorizer=TextVectorization(max_tokens=MAX_FEATURES, #number of words in the vocab
                            output_sequence_length=1800, #length of a sentence
                            output_mode='int') #output mode must be integer

x below is a series object of Panda Series. It contains the text data from 'comment_text" column.


In [ ]:
type(x)

.value attribute converts the data into numPy array format.


---

numpy.ndarray stands for N dimensional array



In [ ]:
type(x.values)

In [ ]:
vectorizer.adapt(x.values)

In [ ]:
vectorizer("Hello World, How are you ? hope you are fine")[:10]

In [ ]:
vectorizer.get_vocabulary()

In [ ]:
vectorizer.adapt(x.values)

In [ ]:
vectorized_text=vectorizer(x.values)

In [ ]:
len(x)

In [ ]:
vectorized_text

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset =dataset.cache()
dataset = dataset.shuffle(160000) #buffer size 160000
dataset = dataset.batch(16)
dataset = dataset.prefetch(8) # helps prevent bottlenecks

In [ ]:
batch_x,batch_y=dataset.as_numpy_iterator().next()

16 comments each of length 1800


In [ ]:
batch_x.shape

16 correcponding labels with respect to x
16 samples in the batch, each with 6 label categories

In [ ]:
batch_y.shape


Total number of batches in the dataset

In [ ]:
len(dataset) #length in batches

In [ ]:
train = dataset.take(int(len(dataset)*0.7)) # 70% of total data
val = dataset.skip(int(len(dataset)*0.7)).take(int(len(dataset)*0.2)) #.skip will skip the 70% data and then get another 20% data
test = dataset.skip(int(len(dataset)*0.9)).take(int(len(dataset)*0.1))

In [ ]:
len(train)

In [ ]:
len(val)

In [ ]:
len(test)

onverts the TensorFlow dataset train into a NumPy iterator

In [ ]:
train_generator = train.as_numpy_iterator()

In [ ]:
train_generator.next()

In [ ]:
#create a Sequential Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding #layers used to build deep learning layers

In [ ]:
model = Sequential() #instantiate the sequential api to add the various layers to the model
#create the embedding layer
model.add(Embedding(MAX_FEATURES+1, 32)) # +1 is for unknown words
# one embedding per word
#Bidirectional passes the info in both directions backward and forward
model.add(Bidirectional(LSTM(32, activation='tanh')))  # 32 different LSTM units
# feature extractor of fully cocnnected layers
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
#final output layer converts value between 0 to 1
model.add(Dense(6, activation='sigmoid'))

In [ ]:
model.compile(loss='BinaryCrossentropy', optimizer='adam',metrics=['accuracy'])  #configure learning process

In [ ]:
model.summary()

In [ ]:
history = model.fit(train, epochs=3, validation_data=val)  #train the model

Epoch 1/3
6981/6981 ━━━━━━━━━━━━━━━━━━━━ 11885s 2s/step - accuracy: 0.9426 - loss: 0.0837 - val_accuracy: 0.9933 - val_loss: 0.0449
Epoch 2/3
6110/6981 ━━━━━━━━━━━━━━━━━━━━ 24:01 2s/step - accuracy: 0.9862 - loss: 0.0455

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.figure(figsize=(8,5))
pd.DataFrame(history.history).plot()
plt.show()

In [ ]:
input_text=vectorizer('You freaking suck')

In [ ]:
batch = test.as_numpy_iterator().next()

In [ ]:
test.as_numpy_iterator().next()

In [ ]:
model.predict(np.expand_dims(input_text,0 ))

In [ ]:
res =model.predict(np.expand_dims(input_text,0 ))

In [ ]:
(res > 0.5).astype(int)

In [ ]:
batch_X, batch_y = test.as_numpy_iterator().next()

In [ ]:
(model.predict(batch_X) > 0.5).astype(int)

In [ ]:
res.shape

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [ ]:
for batch in test.as_numpy_iterator():
    # Unpack the batch
    X_true, y_true = batch
    # Make a prediction
    yhat = model.predict(X_true)

    # Flatten the predictions
    y_true = y_true.flatten()
    yhat = yhat.flatten()

    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

In [ ]:
print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

In [ ]:
!pip install gradio jinja2

In [ ]:
import tensorflow as tf
import gradio as gr

In [ ]:
model.save('toxicity.h5')

In [ ]:
model = tf.keras.models.load_model('toxicity.h5')

In [ ]:
input_str = vectorizer('hey i freaken hate you!')

In [ ]:
res = model.predict(np.expand_dims(input_str,0))

In [ ]:
res

In [ ]:
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)

    text = ''
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)

    return text

In [ ]:
interface = gr.Interface(fn=score_comment,
                         inputs=gr.inputs.Textbox(lines=2, placeholder='Comment to score'),
                        outputs='text')

In [ ]:
interface.launch(share=True)